<a href="https://colab.research.google.com/github/shige-ta/WeatherBinaryClassifier/blob/main/classify_weather.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# データセットをダウンロード
!wget https://prod-dcd-datasets-cache-zipfiles.s3.eu-west-1.amazonaws.com/4drtyfjtfy-1.zip -O /content/dataset2.zip

# ダウンロードしたzipファイルを解凍
!unzip /content/dataset2.zip -d /content/test/
!unzip /content/test/dataset2.zip



--2023-07-15 09:30:48--  https://prod-dcd-datasets-cache-zipfiles.s3.eu-west-1.amazonaws.com/4drtyfjtfy-1.zip
Resolving prod-dcd-datasets-cache-zipfiles.s3.eu-west-1.amazonaws.com (prod-dcd-datasets-cache-zipfiles.s3.eu-west-1.amazonaws.com)... 52.218.61.0, 3.5.64.12, 52.218.91.128, ...
Connecting to prod-dcd-datasets-cache-zipfiles.s3.eu-west-1.amazonaws.com (prod-dcd-datasets-cache-zipfiles.s3.eu-west-1.amazonaws.com)|52.218.61.0|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 95592747 (91M) [application/octet-stream]
Saving to: ‘/content/dataset2.zip’

/content/dataset2.z 100%[===================>]  91.16M  19.0MB/s    in 5.7s    

2023-07-15 09:30:55 (16.0 MB/s) - ‘/content/dataset2.zip’ saved [95592747/95592747]

Archive:  /content/dataset2.zip
replace /content/test/dataset2.zip? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: /content/test/dataset2.zip  A

Archive:  /content/test/dataset2.zip
replace dataset2/cloudy1.jpg? [y]es, [n]o, [A]ll, [N]one,

In [2]:
!mkdir -p /content/dataset2/train/sunrise/
!mkdir -p /content/dataset2/train/shine/
!mkdir -p /content/dataset2/train/cloudy/
!mkdir -p /content/dataset2/train/rain/



!mv /content/dataset2/sunrise*.* /content/dataset2/train/sunrise/
!mv /content/dataset2/shine*.* /content/dataset2/train/shine/
!mv /content/dataset2/cloudy*.* /content/dataset2/train/cloudy/
!mv /content/dataset2/rain*.* /content/dataset2/train/rain/


In [1]:
from torchvision import models, transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, random_split
import torch.nn as nn
import torch.optim as optim
import torch

# データセットの場所
data_dir = "/content/dataset2/train"

# 入力画像のサイズ（このサイズに全ての画像がリサイズされます）
input_size = 224

# バッチサイズ（一度に何枚の画像を処理するか）
batch_size = 64

# 学習のエポック数（データセットを何回繰り返して学習するか）
num_epochs = 25

# 画像の前処理を定義
data_transforms = transforms.Compose([
    transforms.RandomResizedCrop(input_size),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# 全体のデータセットを作成
full_dataset = ImageFolder(data_dir, transform=data_transforms)

# 訓練データと検証データに分割
train_size = int(0.8 * len(full_dataset))  # 80%を訓練データとする
val_size = len(full_dataset) - train_size
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

# DataLoaderを作成
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True, num_workers=4)

# DataLoaderの辞書を作成
dataloaders_dict = {'train': train_loader, 'val': val_loader}

# モデルを定義（ここではResNetを使用）
model = models.resnet50(pretrained=True)

# 全結合層の出力をクラス数に合わせて変更
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 4)

# デバイスを定義（GPUが利用可能ならGPUを使用）
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# モデルをデバイスに移動
model = model.to(device)

# 損失関数と最適化手法を定義
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# 学習ループ
for epoch in range(num_epochs):
    print(f"Epoch {epoch}/{num_epochs - 1}")
    print("-" * 10)

    for phase in ['train', 'val']:
        if phase == 'train':
            model.train()
        else:
            model.eval()

        running_loss = 0.0
        running_corrects = 0

        for inputs, labels in dataloaders_dict[phase]:
            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()

            with torch.set_grad_enabled(phase == 'train'):
                outputs = model(inputs)
                loss = criterion(outputs, labels)

                _, preds = torch.max(outputs, 1)

                if phase == 'train':
                    loss.backward()
                    optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        epoch_loss = running_loss / len(dataloaders_dict[phase].dataset)
        epoch_acc = running_corrects.double() / len(dataloaders_dict[phase].dataset)

        print(f"{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}")

print("Training complete")

# モデルを保存
torch.save(model.state_dict(), "/content/weather_classifier_model.pth")


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can als

Epoch 0/24
----------
train Loss: 1.2210 Acc: 0.4922
val Loss: 0.8986 Acc: 0.6978
Epoch 1/24
----------
train Loss: 0.6544 Acc: 0.8833
val Loss: 0.4311 Acc: 0.9067
Epoch 2/24
----------
train Loss: 0.3090 Acc: 0.9333
val Loss: 0.2563 Acc: 0.9333
Epoch 3/24
----------
train Loss: 0.2076 Acc: 0.9389
val Loss: 0.1462 Acc: 0.9644
Epoch 4/24
----------
train Loss: 0.1450 Acc: 0.9656
val Loss: 0.1552 Acc: 0.9556
Epoch 5/24
----------
train Loss: 0.1578 Acc: 0.9478
val Loss: 0.1459 Acc: 0.9511
Epoch 6/24
----------
train Loss: 0.1561 Acc: 0.9478
val Loss: 0.2017 Acc: 0.9289
Epoch 7/24
----------
train Loss: 0.1419 Acc: 0.9511
val Loss: 0.1313 Acc: 0.9556
Epoch 8/24
----------
train Loss: 0.1250 Acc: 0.9633
val Loss: 0.1303 Acc: 0.9556
Epoch 9/24
----------
train Loss: 0.1126 Acc: 0.9622
val Loss: 0.1338 Acc: 0.9689
Epoch 10/24
----------
train Loss: 0.0862 Acc: 0.9733
val Loss: 0.0907 Acc: 0.9689
Epoch 11/24
----------
train Loss: 0.1063 Acc: 0.9700
val Loss: 0.0960 Acc: 0.9689
Epoch 12/24
--

In [5]:
from PIL import Image
import torch
from torchvision import transforms, models
import torch.nn.functional as F

# 画像のパス
image_path = "/content/not_sunny.png"

# モデルを読み込み
model = models.resnet50(pretrained=False)
num_ftrs = model.fc.in_features
model.fc = torch.nn.Linear(num_ftrs, 4)
model.load_state_dict(torch.load("/content/weather_classifier_model.pth"))
model = model.to(device)
model.eval()

# デバイスを定義（GPUが利用可能ならGPUを使用）
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# 画像の前処理を定義
transform = transforms.Compose([
    transforms.Resize((input_size, input_size)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# 画像を読み込み、前処理
image = Image.open(image_path)
# RGBA画像をRGBに変換
if image.mode == 'RGBA':
    image = image.convert('RGB')

image = transform(image).unsqueeze(0)

# 画像をデバイスに移動
image = image.to(device)

# モデルで画像を判定
outputs = model(image)
_, preds = torch.max(outputs, 1)

# 出力を確率に変換
probs = F.softmax(outputs, dim=1)

# 判定結果と評価値を表示
print(f"The image is classified as: {full_dataset.classes[preds.item()]}")
print(f"Probabilities: {probs.cpu().detach().numpy()}")



The image is classified as: rain
Probabilities: [[4.4231721e-05 9.9990022e-01 2.8785589e-05 2.6849217e-05]]
